# GREAT CARIA v3.0 - Full ML Pipeline

- Grid search hyperparameter tuning
- Multiple models (LR, RF, XGBoost)
- Systematic feature ablation with L1/L2
- Multiple horizons (5, 10, 20 days)

In [ ]:
!pip install xgboost -q

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print('Setup complete')

In [ ]:
# === LOAD DATA ===
from google.colab import drive
drive.mount('/content/drive')

MARKET_PATH = '/content/drive/MyDrive/CARIA/data/raw/yahoo_market.parquet'
df = pd.read_parquet(MARKET_PATH)
COUNTRIES = ['USA', 'CHN', 'JPN', 'DEU', 'GBR', 'FRA', 'BRA', 'MEX', 'KOR', 'AUS']
print(f'Loaded: {df.shape}')

In [ ]:
# === CRISIS FACTOR ===
idx_cols = [f'{c}_index' for c in COUNTRIES if f'{c}_index' in df.columns]
ret = df[idx_cols].pct_change().dropna()
ret.columns = [c.replace('_index', '') for c in ret.columns]

def compute_cf(r, w=20):
    cf = []
    for i in range(w, len(r)):
        wr = r.iloc[i-w:i]
        c = wr.corr().values
        ac = (c.sum() - len(c)) / (len(c) * (len(c) - 1))
        cf.append(ac * wr.std().mean() * 100)
    return pd.Series(cf, index=r.index[w:])

CF = compute_cf(ret)
print(f'CF: {len(CF)} samples')

In [ ]:
# === FEATURES (extended) ===
def build_features(CF, df, ret):
    f = pd.DataFrame(index=CF.index)
    
    # CF features
    f['cf'] = CF
    f['cf_ma5'] = CF.rolling(5).mean()
    f['cf_ma20'] = CF.rolling(20).mean()
    f['cf_std5'] = CF.rolling(5).std()
    f['cf_change5'] = CF.diff(5)
    
    # Market features
    f['vix'] = df['VIX'].loc[CF.index]
    f['vix_ma5'] = df['VIX'].rolling(5).mean().loc[CF.index]
    f['dxy'] = df['DXY'].loc[CF.index]
    
    # Returns features
    for c in ['USA', 'BRA', 'CHN']:
        if c in ret.columns:
            f[f'ret_{c}_5'] = ret[c].rolling(5).mean().loc[CF.index]
            f[f'ret_{c}_20'] = ret[c].rolling(20).mean().loc[CF.index]
    
    return f.dropna()

features = build_features(CF, df, ret)
print(f'Features: {features.shape}')
print(f'Columns: {list(features.columns)}')

In [ ]:
# === TARGETS: Multiple horizons ===
HORIZONS = [5, 10, 20]
targets = {}
for h in HORIZONS:
    cf_future = CF.shift(-h)
    targets[h] = (cf_future > CF).astype(int).loc[features.index].dropna()
    print(f'H={h}: {len(targets[h])} samples, balance={targets[h].mean():.1%}')

In [ ]:
# === PURGED TEMPORAL SPLIT ===
PURGE, EMBARGO = 20, 10

def purged_split(X, y, train_pct=0.7):
    n = len(X)
    train_end = int(n * train_pct)
    test_start = train_end + PURGE + EMBARGO
    
    X_train, y_train = X.iloc[:train_end], y.iloc[:train_end]
    X_test, y_test = X.iloc[test_start:], y.iloc[test_start:]
    
    # Normalize
    mu, std = X_train.mean(), X_train.std() + 1e-8
    X_train = (X_train - mu) / std
    X_test = (X_test - mu) / std
    
    return X_train.values, X_test.values, y_train.values, y_test.values

In [ ]:
# === MODEL DEFINITIONS ===
models = {
    'LogReg_L1': LogisticRegression(penalty='l1', solver='saga', max_iter=1000),
    'LogReg_L2': LogisticRegression(penalty='l2', max_iter=1000),
    'RandomForest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, max_depth=3, random_state=42, verbosity=0)
}

# Grid search params
param_grids = {
    'LogReg_L1': {'C': [0.01, 0.1, 1.0, 10.0]},
    'LogReg_L2': {'C': [0.01, 0.1, 1.0, 10.0]},
    'RandomForest': {'n_estimators': [50, 100], 'max_depth': [3, 5, 7]},
    'GradientBoosting': {'n_estimators': [50, 100], 'max_depth': [2, 3, 5]},
    'XGBoost': {'n_estimators': [50, 100], 'max_depth': [2, 3, 5]}
}

In [ ]:
# === RUN EXPERIMENTS ===
results = []

for h in HORIZONS:
    print(f'\n=== HORIZON {h} days ===')
    y = targets[h]
    X = features.loc[y.index]
    
    X_train, X_test, y_train, y_test = purged_split(X, y)
    print(f'Train: {len(X_train)}, Test: {len(X_test)}')
    
    for name, model in models.items():
        # Grid search with time series CV
        tscv = TimeSeriesSplit(n_splits=3)
        grid = GridSearchCV(model, param_grids[name], cv=tscv, scoring='accuracy', n_jobs=-1)
        grid.fit(X_train, y_train)
        
        best = grid.best_estimator_
        y_pred = best.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        
        # Shuffle test
        y_shuffled = np.random.permutation(y_train)
        best_shuffle = grid.best_estimator_.__class__(**grid.best_params_)
        best_shuffle.fit(X_train, y_shuffled)
        acc_shuffle = accuracy_score(y_test, best_shuffle.predict(X_test))
        
        results.append({
            'horizon': h,
            'model': name,
            'accuracy': acc,
            'shuffle_acc': acc_shuffle,
            'lift': acc - acc_shuffle,
            'best_params': grid.best_params_
        })
        
        print(f'{name}: {acc:.1%} (shuffle: {acc_shuffle:.1%}, lift: {(acc-acc_shuffle)*100:.1f}pp)')

In [ ]:
# === FEATURE ABLATION (L1 regularization) ===
print('\n=== FEATURE IMPORTANCE (L1) ===')

h = 10  # Use 10-day horizon
y = targets[h]
X = features.loc[y.index]
X_train, X_test, y_train, y_test = purged_split(X, y)

# Fit L1 model
l1 = LogisticRegressionCV(penalty='l1', solver='saga', cv=3, max_iter=1000)
l1.fit(X_train, y_train)

# Feature importance
importance = pd.DataFrame({
    'feature': features.columns,
    'coef': np.abs(l1.coef_[0])
}).sort_values('coef', ascending=False)

print('\nTop features:')
print(importance.head(10).to_string(index=False))

# Ablation: remove top features one by one
print('\nAblation test:')
base_acc = accuracy_score(y_test, l1.predict(X_test))
print(f'Full model: {base_acc:.1%}')

for feat in importance['feature'].head(5):
    X_ablated = X.drop(columns=[feat])
    X_tr, X_te, y_tr, y_te = purged_split(X_ablated, y)
    l1_abl = LogisticRegression(penalty='l1', solver='saga', C=l1.C_[0], max_iter=1000)
    l1_abl.fit(X_tr, y_tr)
    abl_acc = accuracy_score(y_te, l1_abl.predict(X_te))
    print(f'  Without {feat}: {abl_acc:.1%} (drop: {(base_acc-abl_acc)*100:.1f}pp)')

In [ ]:
# === SUMMARY ===
print('\n' + '='*60)
print('GREAT CARIA v3.0 - FULL RESULTS')
print('='*60)

df_results = pd.DataFrame(results)
print('\nAll experiments:')
print(df_results.to_string(index=False))

# Best result
best = df_results.loc[df_results['accuracy'].idxmax()]
print(f'\n🏆 Best: {best["model"]} @ H={best["horizon"]}d')
print(f'   Accuracy: {best["accuracy"]:.1%}')
print(f'   Lift: {best["lift"]*100:.1f}pp')

# Verdict
if best['accuracy'] > 0.55 and best['lift'] > 0.03:
    print('\n✓ PASSED - Real signal detected')
else:
    print('\n✗ FAILED - Signal is marginal')

In [ ]:
# === VISUALIZATION ===
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy by horizon
for h in HORIZONS:
    subset = df_results[df_results['horizon'] == h]
    axes[0].bar([f'{m}\n(H={h})' for m in subset['model']], subset['accuracy'])
axes[0].axhline(0.5, color='red', linestyle='--', label='Random')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy by Model & Horizon')
axes[0].tick_params(axis='x', rotation=45)

# Lift by model
pivot = df_results.pivot(index='model', columns='horizon', values='lift')
pivot.plot(kind='bar', ax=axes[1])
axes[1].axhline(0.03, color='green', linestyle='--', label='Min lift (3%)')
axes[1].set_ylabel('Lift over shuffle')
axes[1].set_title('Lift by Model & Horizon')
axes[1].legend()

plt.tight_layout()
plt.show()